In [1]:
import numpy as np
import pandas as pd

from src.M4_data import prepare_M4_data
from src.M4_experiment import plot_model_prediction, evaluate_model_prediction
from src.ESRNN import ESRNN

In [2]:
X_train_df, y_train_df, X_test_df, y_test_df = prepare_M4_data('Hourly', num_obs=3) #, num_obs=5
y_train_df

,unique_id,ds,y
0,H1,1970-01-02,605.0
1,H1,1970-01-03,586.0
2,H1,1970-01-04,586.0
3,H1,1970-01-05,559.0
4,H1,1970-01-06,511.0
...,...,...,...
2095,H3,1971-11-28,1517.0
2096,H3,1971-11-29,1570.0
2097,H3,1971-11-30,1581.0
2098,H3,1971-12-01,1570.0


In [3]:
import torch

## Weekly
# esrnn = ESRNN(max_epochs=4, batch_size=4, dilations=[[1, 4], [52]], rnn_weight_decay=0.5, freq_of_test=1)

## Quarterly
# esrnn = ESRNN(max_epochs=1, batch_size=3, rnn_weight_decay=0.5, freq_of_test=1)

## Daily
# esrnn = ESRNN(max_epochs=2, batch_size=16, learning_rate=3e-4, per_series_lr_multip=1.5,
#               gradient_eps=1e-6, level_variability_penalty=100,
#               dilations=[[1,7],[28]], add_nl_layer=True,
#               seasonality=[7], input_size=7, output_size=14)

## Debugging Quarterly
# with torch.autograd.detect_anomaly():
#     esrnn = ESRNN(max_epochs=4, max_periods=1, batch_size=3, state_hsize=4,
#                   seasonality=[1], input_size=3, output_size=8, dilations=[[1, 2]],
#                   rnn_weight_decay=1.0, freq_of_test=1, random_seed=1, cell_type='LSTM')
#     esrnn.fit(X_train_df, y_train_df, X_test_df, y_test_df)
    
## Debugging Yearly
# with torch.autograd.detect_anomaly():
#     esrnn = ESRNN(max_epochs=4, max_periods=1, batch_size=3, state_hsize=4,
#                   seasonality=[24, 168], input_size=3, output_size=6, dilations=[[1, 2]],
#                   rnn_weight_decay=1.0, freq_of_test=1, random_seed=1, cell_type='LSTM')
#     esrnn.fit(X_train_df, y_train_df, X_test_df, y_test_df)

## Debugging Hourly
with torch.autograd.detect_anomaly():
    esrnn = ESRNN(max_epochs=4, max_periods=1, batch_size=3, state_hsize=4,
                  seasonality=[24, 168], input_size=3, output_size=48, dilations=[[1, 2]],
                  rnn_weight_decay=1.0, freq_of_test=1, random_seed=1, cell_type='LSTM')
    esrnn.fit(X_train_df, y_train_df, X_test_df, y_test_df)

=============== Training ESRNN  ===============

========= Epoch 0 finished =========
Training time: 0.76636
Training loss: 0.18889999389648438
OWA: 6.551 
SMAPE: 33.472 
MASE: 6.325 
========= Epoch 1 finished =========
Training time: 0.76119
Training loss: 0.18535999953746796
OWA: 6.539 
SMAPE: 33.421 
MASE: 6.311 
========= Epoch 2 finished =========
Training time: 0.73225
Training loss: 0.18241000175476074
OWA: 6.526 
SMAPE: 33.368 
MASE: 6.296 
========= Epoch 3 finished =========
Training time: 0.72678
Training loss: 0.1798900067806244
OWA: 6.512 
SMAPE: 33.313 
MASE: 6.28 
Train finished! 



In [ ]:
plot_model_prediction(y_train_df, X_test_df, y_test_df, model=esrnn, u_id='Q1')

In [ ]:
esrnn.evaluate_model_prediction(y_train_df, X_test_df, y_test_df)

In [ ]:
# M4 eval
#(0.778*(23.0/100) + 0.847*(24.0/100) + 0.836*(48.0/100) + 0.920*(5.0/100))
#(13.176*() + 9.679*() + 12.126*())
#(14.42*(23.0/(23+24+48)) + 10.09*(24.0/(23+24+48)) + 10.81*(48.0/(23+24+48)))

In [ ]:
from src.hyperpar_tunning_m4 import parse_grid_search
from src.utils_visualization import plot_grid_cat_distributions

gs_df = parse_grid_search('Quarterly')
# plot_cat_distributions(df=gs_df, cat='learning_rate', var='min_owa')
# plot_cat_distributions(df=gs_df, cat='add_nl_layer', var='min_owa')
# plot_cat_distributions(df=gs_df, cat='rnn_weight_decay', var='min_owa')
# plot_cat_distributions(df=gs_df, cat='per_series_lr_multip', var='min_owa')
# plot_cat_distributions(df=gs_df, cat='batch_size', var='min_owa')
# plot_cat_distributions(df=gs_df, cat='training_percentile', var='min_owa')
# plot_cat_distributions(df=gs_df, cat='dilations', var='min_owa')

gs_df['early_stopping'] = gs_df.min_epoch < gs_df.max_epochs
plot_grid_cat_distributions(gs_df, var = 'min_owa',
                            cats=['learning_rate', 'per_series_lr_multip', 'batch_size', 'early_stopping',
                                  'rnn_weight_decay', 'training_percentile', 'add_nl_layer', 'dilations'])
# plot_grid_cat_distributions(gs_df, var = 'owa',
#                             cats=['learning_rate', 'per_series_lr_multip', 'batch_size', 'early_stopping',
#                                   'rnn_weight_decay', 'training_percentile', 'add_nl_layer', 'dilations'])

In [ ]:
gs_df = parse_grid_search('Monthly')
gs_df.min_owa.max()

In [ ]:
gs_df = parse_grid_search('Quarterly')
gs_df.min_owa.max()

In [ ]:
ver = esrnn.predict(X_test_df)

In [ ]:
import pandas as pd

pd.set_option('display.max_rows', 100)

In [ ]:
for row in ver.iterrows():
    print(row)